Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.

##### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Intel® Gaudi® Accelerator Demo 


This document provides instructions on setting up the Intel Gaudi 2 AI accelerator Instance on the Intel® Developer Cloud or any on-premise Intel Gaudi Node. You will be running models from the Intel Gaudi software Model References and the Hugging Face Optimum Habana library.

Please follow along with the [video](https://developer.habana.ai/intel-developer-cloud/) on our Developer Page to walk through the steps below.  This assumes that you have setup the latest Intel Gaudi PyTorch Docker image.

To set up a multi-node instance with two or more Gaudi nodes, refer to Setting up Multiple Gaudi Nodes in the [Quick Start Guide Documentation](https://docs.habana.ai/en/latest/Intel_DevCloud_Quick_Start/Intel_DevCloud_Quick_Start.html#setting-up-multiple-gaudi-nodeshttps://docs.habana.ai/en/latest/Intel_DevCloud_Quick_Start/Intel_DevCloud_Quick_Start.html#setting-up-multiple-gaudi-nodes).  

The first step is to install the Optimum Habana repository from GitHub and run the demo of Transformer Reinforcement Learning.

## Fine-tuning with Hugging Face Optimum Habana Library
The Optimum Habana library is the interface between the Hugging Face Transformers and Diffusers libraries and the Gaudi 2 card. It provides a set of tools enabling easy model loading, training and inference on single and multi-card settings for different downstream tasks. The following example use the DPO and PPO pipeline to fine-tune a Llama 2 7B model.

Follow the below steps to install the stable release from the Optimum Habana examples and library:

1. Clone the Optimum-Habana project and check out the lastest stable release.  This repository gives access to the examples that are optimized for Intel Gaudi:

In [ ]:
%cd ~
!git clone https://github.com/huggingface/optimum-habana.git
%cd optimum-habana
!git checkout v1.11.1
%cd ~

2. Install Optimum-Habana library. This will install the latest stable library:

In [ ]:
!pip install optimum-habana==1.11.1

3. In order to use the DeepSpeed library on Intel Gaudi 2, install the Intel Gaudi DeepSpeed fork:

In [ ]:
!pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.15.0

The following example is based on the Optimum-Habana Text Classification task example. Change to the text-classification directory and install the additional SW requirements for this specific example:

In [ ]:
%cd ~/optimum-habana/examples/trl/
!pip install -U -r requirements.txt
!pip install datasets==2.18

In [ ]:
!huggingface-cli login --token <your_token_here>

### DPO Pipeline

#### Training

The following example is for the creation of StackLlaMa 2: a Stack exchange llama-v2-7b model. There are two main steps to the DPO training process:

1. Supervised fine-tuning of the base llama-v2-7b model to create llama-v2-7b-se:

In [ ]:
!python sft.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --output_dir="./sft" \
    --max_steps=500 \
    --logging_steps=10 \
    --save_steps=100 \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=1 \
    --gradient_accumulation_steps=2 \
    --learning_rate=1e-4 \
    --lr_scheduler_type="cosine" \
    --warmup_steps=100 \
    --weight_decay=0.05 \
    --optim="paged_adamw_32bit" \
    --lora_target_modules "q_proj" "v_proj" \
    --bf16 \
    --remove_unused_columns=False \
    --run_name="sft_llama2" \
    --report_to=none \
    --use_habana \
    --use_lazy_mode

2. Run the DPO trainer using the model saved by the previous step:

In [ ]:
!python dpo.py \
    --model_name_or_path="sft/final_merged_checkpoint" \
    --tokenizer_name_or_path=meta-llama/Llama-2-7b-hf \
    --lora_target_modules "q_proj" "v_proj" "k_proj" "out_proj" "fc_in" "fc_out" "wte" \
    --output_dir="dpo" \
    --report_to=none

#### Merging the adaptors

To merge the adaptors into the base model we can use the merge_peft_adapter.py helper script that comes with TRL:

In [ ]:
!python merge_peft_adapter.py --base_model_name="meta-llama/Llama-2-7b-hf" --adapter_model_name="dpo" --output_name="stack-llama-2"

which will also push the model to your HuggingFace hub account.

#### Running the model

We can load the DPO-trained LoRA adaptors which were saved by the DPO training step and run it through the [text-generation example]([../text-generation/](https://github.com/huggingface/optimum-habana/tree/main/examples/text-generation)).

In [ ]:
%cd ~/optimum-habana/examples/text-generation/
!python run_generation.py \
--model_name_or_path ../trl/stack-llama-2/ \
--use_hpu_graphs --use_kv_cache --batch_size 1 --bf16 --max_new_tokens 100 \
--prompt "Here is my prompt"

## PPO Pipeline

### Training

The following example is for the creation of StackLlaMa 2: a Stack exchange llama-v2-7b model. There are three main steps to the PPO training process:

1. Supervised fine-tuning of the base llama-v2-7b model to create llama-v2-7b-se:

In [ ]:
%cd ~/optimum-habana/examples/trl/
!python sft.py \
    --model_name_or_path meta-llama/Llama-2-7b-hf \
    --output_dir="./sft" \
    --max_steps=500 \
    --logging_steps=10 \
    --save_steps=100 \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=1 \
    --gradient_accumulation_steps=2 \
    --learning_rate=1e-4 \
    --lr_scheduler_type="cosine" \
    --warmup_steps=100 \
    --weight_decay=0.05 \
    --optim="paged_adamw_32bit" \
    --lora_target_modules "q_proj" "v_proj" \
    --bf16 \
    --remove_unused_columns=False \
    --run_name="sft_llama2" \
    --report_to=none \
    --use_habana \
    --use_lazy_mode

2. Reward modeling using dialog pairs from the SE dataset on the llama-v2-7b-se to create llama-v2-7b-se-rm

In [ ]:
!python reward_modeling.py \
    --model_name=./sft/final_merged_checkpoint \
    --tokenizer_name=meta-llama/Llama-2-7b-hf \
    --output_dir=./rm

To merge the adaptors into the base model we can use the `merge_peft_adapter.py` helper script that comes with TRL:

In [ ]:
!python merge_peft_adapter.py --base_model_name="meta-llama/Llama-2-7b-hf" --adapter_model_name="rm" --output_name="rm_merged_checkpoint"

3. RL fine-tuning of llama-v2-7b-se with the llama-v2-7b-se-rm reward model:

In [ ]:
!python ppo.py \
    --model_name=./sft/final_merged_checkpoint \
    --reward_model_name=./rm_merged_checkpoint \
    --tokenizer_name=meta-llama/Llama-2-7b-hf \
    --adafactor=False \
    --output_max_length=128 \
    --batch_size=8 \
    --gradient_accumulation_steps=8 \
    --batched_gen=True \
    --ppo_epochs=4 \
    --seed=0 \
    --learning_rate=1.4e-5 \
    --early_stopping=True \
    --output_dir=llama-se-rl-finetune

To merge the adaptors into the base model we can use the merge_peft_adapter.py helper script that comes with TRL:

In [ ]:
!python merge_peft_adapter.py --base_model_name="meta-llama/Llama-2-7b-hf" --adapter_model_name="llama-se-rl-finetune" --output_name="rl_merged_checkpoint"

### Running the model

We can load the PPO-trained LoRA adaptors which were saved by the PPO training step and run it through the [text-generation example](https://github.com/huggingface/optimum-habana/tree/main/examples/text-generation).

In [ ]:
%cd ~/optimum-habana/examples/text-generation/
!python run_generation.py \
--model_name_or_path ../trl/rl_merged_checkpoint/ \
--use_hpu_graphs --use_kv_cache --batch_size 1 --bf16 --max_new_tokens 100 \
--prompt "Here is my prompt"

## Next Steps
You now have access to all the Models in Model-References and Optimum-Habana repositories, you can start to look at other models.  Remember that all the models in these repositories are fully documented so they are easy to use.
* To explore more models from the Model References, start [here](https://github.com/HabanaAI/Model-References).  
* To run more examples using Hugging Face go [here](https://github.com/huggingface/optimum-habana?tab=readme-ov-file#validated-models).  
* To migrate other models to Gaudi 2, refer to PyTorch Model Porting in the [documentation](https://docs.habana.ai/en/latest/PyTorch/PyTorch_Model_Porting/GPU_Migration_Toolkit/GPU_Migration_Toolkit.html)